In [ ]:
# Here are the imports that we will need throughout the all notebook

import pandas as pd
import numpy as np
import os
import time
import matplotlib as plt
import sklearn
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
from ib_insync import *
import asyncio
import scipy
from mpl_toolkits.mplot3d import Axes3D
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

# auto reload to reload functions imported that have been changed (cf. test_functions.sphereL1 for lbda)
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
ib = IB()
util.startLoop()
await ib.connectAsync('127.0.0.1', 7497, clientId=4) # Don't forget to change the client ID

In [ ]:
P = {"TTE": {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'AIR': {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'EXA': {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'MC': {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'ORA': {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'GLE': {"exchange": "SMART", "currency": "EUR", "primary": "SBF"},
     'ORCL': {"exchange": "SMART", "currency": "USD", "primary": "NASDAQ"},
     'BA': {"exchange": "SMART", "currency": "USD", "primary": "NASDAQ"},
     'COST': {"exchange": "SMART", "currency": "USD", "primary": "NASDAQ"},
     'NVDA': {"exchange": "SMART", "currency": "USD", "primary": "NASDAQ"},
     'WMT': {"exchange": "SMART", "currency": "USD", "primary": "NASDAQ"}
    }


In [ ]:
async def fetch_stock_df(
    ib,
    symbol,
    exchange,
    currency,
    primaryExchange,
    duration='10 Y',
    barSize='1 day'
):
    contract = Stock(
        symbol=symbol,
        exchange=exchange,
        currency=currency,
        primaryExchange=primaryExchange
    )

    await ib.qualifyContractsAsync(contract)

    bars = await ib.reqHistoricalDataAsync(
        contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=barSize,
        whatToShow='TRADES',
        useRTH=True
    )

    df = util.df(bars)
    df = df.set_index('date')[['close']]
    df.rename(columns={'close': symbol}, inplace=True)

    return df

async def fetch_one_symbol(ib, symbol, cfg):
    try:
        return await fetch_stock_df(
            ib=ib,
            symbol=symbol,
            exchange=cfg.get("exchange", "SMART"),
            currency=cfg.get("currency", "USD"),
            primaryExchange=cfg.get("primary", None)
        )
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return None

async def fetch_portfolio_prices(ib, P):
    tasks = [
        fetch_one_symbol(ib, symbol, cfg)
        for symbol, cfg in P.items()
    ]

    dfs = await asyncio.gather(*tasks)

    dfs = [df for df in dfs if df is not None and not df.empty]

    if not dfs:
        raise ValueError("No data retrieved")

    prices = pd.concat(dfs, axis=1, join="inner")
    prices.sort_index(inplace=True)

    return prices

In [ ]:
prices = await fetch_portfolio_prices(ib, P)


In [ ]:
prices

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm


def BSM_mc_rv(S0, K, T, r, q, sigma, option, N, seed = None):

    """

    S0 : le prix spot
    K le prix strike
    T : la maturité
    q : le taux de dividende
    sigma : la vol
    option: call ou put
    N: le nombre de simulations effectuées
    
    """
    option = option.lower()
    if option not in ["call", "put"]:
        raise ValueError("L'option doit être 'call' ou 'put'")

    
    if S0 < 0:
        raise ValueError("La valeur du prix spot ne peut pas être négatif")

    if K < 0:
        raise ValueError(" La valeur du strike ne peut pas être négative")

    if T < 0:
        raise ValueError("L'échéance ne peut pas être dans le passé")

    if sigma < 0:
        raise ValueError("La vol ne peut pas etre négative")

    if T==0:
        if option == "call":
            return (max(S0 -K,0))
        if option == "put":
            return (max(K-S0,0))
    ST = S0 * np.exp((r - q) * T)
    if sigma == 0:
        if option == "call":
            return np.exp(-r*T)*(max(ST-K,0))
        else:
            return np.exp(-r*T)*(max(K-ST,0))

    if N <= 0:
        raise ValueError("Le nombre de simulations doit être strictement positif")

    if seed is not None: ##Cette partie permet de fixer pour assurer la reproductibilité
        np.random.seed(seed)

    Z = np.random.randn(N)
    
    ST_plus = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    ST_minus = S0 * np.exp((r - q - 0.5 * sigma**2) * T - sigma * np.sqrt(T) * Z)

    if option == "call":
        
        payoffs_plus = np.maximum(ST_plus - K, 0)
        payoffs_minus = np.maximum(ST_minus - K, 0)
    
    else:
        
        payoffs_plus = np.maximum(K - ST_plus, 0)
        payoffs_minus = np.maximum(K - ST_minus, 0)

    Y = (1/2) * (payoffs_plus + payoffs_minus)

    X = np.exp(-r*T) * Y

    V = X.mean()
    
    X_mean = X.mean()

    X_square = (X - X_mean)**2


    S_square = N/(N-1)*X_square.mean()

    standard_error = np.sqrt(S_square)/np.sqrt(N)
    

    

    
    return (float(V), float(standard_error))

    